기념일 - 감정 매핑

In [3]:
# 기념일 키워드 → 감정 매핑
event_emotion_map = {
    "졸업": "기쁨",
    "생일": "사랑",
    "결혼": "사랑",
    "발렌타인": "사랑",
    "어버이날": "감사",
    "크리스마스": "기쁨",
    "추석": "감사",
    "설날": "기쁨",
    "장례": "슬픔",
    "추모": "슬픔"
}

def extract_emotion_from_event(event_text: str) -> str:
    for keyword, emotion in event_emotion_map.items():
        if keyword in event_text:
            return emotion
    return "기쁨"  # 기본값

test_events = [
    "민재 졸업식",
    "할머니 장례식",
    "결혼기념일",
    "추석 가족 모임"
]

for e in test_events:
    print(e, "→", extract_emotion_from_event(e))


민재 졸업식 → 기쁨
할머니 장례식 → 슬픔
결혼기념일 → 사랑
추석 가족 모임 → 감사


기념일 - 감정 - 색상 매핑 (감정-색상은 json 저장값 바탕)

In [4]:
import json

# 색상 심리 DB 로드
with open("coloremotion.json", encoding="utf-8") as f:
    color_db = json.load(f)

def recommend_colors_by_emotion(emotion: str, top_k: int = 2):
    # 감정과 일치하는 색상만 필터링
    matched = []
    for color, meta in color_db.items():
        if emotion in meta["emotion"]:
            matched.append((color, meta["percentage"]))
    
    # 퍼센트 기준 내림차순 정렬
    matched.sort(key=lambda x: x[1], reverse=True)
    
    # 상위 top_k 색상 반환
    return [color for color, _ in matched[:top_k]]


# Step 1에서 만든 함수 재사용
events = [
    "민재 졸업식",   # 기쁨
    "결혼기념일",    # 사랑
    "할머니 장례식" # 슬픔
]

for e in events:
    emotion = extract_emotion_from_event(e)
    colors = recommend_colors_by_emotion(emotion, top_k=2)
    print(f"{e} ({emotion}) → 추천 색상: {', '.join(colors)}")



민재 졸업식 (기쁨) → 추천 색상: yellow, orange
결혼기념일 (사랑) → 추천 색상: red, pink
할머니 장례식 (슬픔) → 추천 색상: black


계절 및 색상 다양화

In [ ]:
from datetime import datetime

# 계절별 톤 변환 (단순 매핑 예시)
season_tone_map = {
    "봄": {"yellow": "pastel yellow", "orange": "pastel orange", "red": "pastel red", "pink": "pastel pink"},
    "여름": {"blue": "sky blue", "white": "pure white", "green": "mint green"},
    "가을": {"orange": "burnt orange", "brown": "warm brown", "yellow": "mustard yellow"},
    "겨울": {"red": "deep red", "green": "pine green", "black": "charcoal black"}
}

def get_season(date: datetime) -> str:
    m = date.month
    if 3 <= m <= 5:
        return "봄"
    elif 6 <= m <= 8:
        return "여름"
    elif 9 <= m <= 11:
        return "가을"
    else:
        return "겨울"

def adjust_colors_for_season(colors: list, event_date: datetime) -> list:
    season = get_season(event_date)
    tone_map = season_tone_map.get(season, {})
    adjusted = [tone_map.get(c, c) for c in colors]
    return adjusted


event_date = datetime(2025, 4, 10)

emotion = extract_emotion_from_event("민재 졸업식")
colors = recommend_colors_by_emotion(emotion, top_k=2)
adjusted = adjust_colors_for_season(colors, event_date)

print(f"기본 색상 추천: {colors}")
print(f"계절({get_season(event_date)}) 보정: {adjusted}")



기본 색상 추천: ['yellow', 'orange']
계절(봄) 보정: ['pastel yellow', 'pastel orange']


근거에 기반한 색상

In [9]:
# palette_fetchers.py
import requests

THE_COLOR_API = "https://www.thecolorapi.com/scheme"

def get_scheme_from_thecolorapi(hex_seed: str, mode: str = "complement", count: int = 4) -> list[str]:
    """
    hex_seed: '#733DCB' 혹은 '733DCB' 모두 허용
    mode: 'complement', 'analogic', 'triad', 'tetrad', 'monochrome' 등
    return: HEX 리스트 (예: ['#733DCB', '#CB943D', ...])
    """
    seed = hex_seed.lstrip("#")
    params = {"hex": seed, "mode": mode, "count": count}
    r = requests.get(THE_COLOR_API, params=params, timeout=10)
    r.raise_for_status()
    data = r.json()
    return [c["hex"]["value"] for c in data.get("colors", [])]


In [12]:
import requests

COLORMIND_API = "http://colormind.io/api/"

from typing import Optional

def get_palette_from_colormind(seed_hex: Optional[str] = None) -> list[str]:
    """
    seed_hex를 주면 첫 칸에 seed 고정 후 나머지를 제안(근사). 없으면 랜덤 팔레트.
    return: HEX 리스트 5개
    """
    payload = {"model": "default"}
    if seed_hex:
        # HEX -> [R,G,B]
        seed = seed_hex.lstrip("#")
        rgb = [int(seed[i:i+2], 16) for i in (0,2,4)]
        # seed 하나만 고정, 나머지는 'N' (Colormind 규칙)
        payload["input"] = [rgb, "N", "N", "N", "N"]
    r = requests.post(COLORMIND_API, json=payload, timeout=10)
    r.raise_for_status()
    data = r.json()
    colors = data.get("result", [])
    return [f"#{r:02X}{g:02X}{b:02X}" for r,g,b in colors]


In [13]:
# name2hex.py
NAME2HEX = {
    "yellow": "#FFD32A",
    "pastel yellow": "#FFF3B0",
    "orange": "#F39C12",
    "pastel orange": "#FFD2A6",
    "red": "#E74C3C",
    "deep red": "#8B0000",
    "pink": "#FFC0CB",
    "pastel pink": "#FFD1DC",
    "blue": "#3498DB",
    "sky blue": "#87CEEB",
    "white": "#FFFFFF",
    "pure white": "#FFFFFF",
    "snow white": "#FAFAFA",
    "green": "#2ECC71",
    "mint green": "#98FF98",
    "brown": "#8D6E63",
    "warm brown": "#8B5E3C",
    "mustard yellow": "#D4AF37",
    "charcoal black": "#333333",
    "black": "#000000",
    "pine green": "#01796F",
    # 필요하면 계속 추가
}

def to_hex(color_name: str) -> str:
    return NAME2HEX.get(color_name.lower(), "#733DCB")  # 기본값


In [14]:
# expand_palette.py
from name2hex import to_hex
from palette_fetchers import get_scheme_from_thecolorapi, get_palette_from_colormind

def expand_colors_from_external(seasonal_colors: list[str], use_colormind: bool = False) -> dict:
    """
    seasonal_colors: 계절 보정된 '이름' 리스트 (예: ['pastel yellow','pastel orange'])
    return: {
      'base_hex': ['#FFF3B0','#FFD2A6'],
      'complement': [... up to 3],
      'analogous':  [... up to 3],
      'colormind' : [... optional 5]
    }
    """
    base_hex = [to_hex(c) for c in seasonal_colors if c]
    resp = {"base_hex": base_hex, "complement": [], "analogous": [], "colormind": []}

    if base_hex:
        seed = base_hex[0]
        # 보색 3개
        comp = get_scheme_from_thecolorapi(seed, mode="complement", count=4)
        resp["complement"] = [h for h in comp if h.upper() != seed.upper()][:3]
        # 유사색 3개
        anag = get_scheme_from_thecolorapi(seed, mode="analogic", count=5)
        resp["analogous"] = [h for h in anag if h.upper() != seed.upper()][:3]
        # (선택) Colormind 감성 팔레트
        if use_colormind:
            resp["colormind"] = get_palette_from_colormind(seed)
    return resp


ModuleNotFoundError: No module named 'name2hex'

In [ ]:
# hybrid_recommender.py
import os, json, re
from datetime import datetime
from typing import Optional
from dotenv import load_dotenv
import google.generativeai as genai

load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# --- 0) 데이터 로드 ---
with open("coloremotion.json", encoding="utf-8") as f:
    COLOR_DB = json.load(f)

# --- 1) 규칙 기반: 기념일 → 감정 ---
EVENT_EMOTION_MAP = {
    "졸업": "기쁨",
    "생일": "사랑",
    "결혼": "사랑",
    "기념일": "사랑",
    "발렌타인": "사랑",
    "어버이날": "감사",
    "스승의날": "감사",
    "추석": "감사",
    "설날": "기쁨",
    "크리스마스": "기쁨",
    "장례": "슬픔",
    "제사": "슬픔",
    "추모": "슬픔",
}
def extract_emotion_rule(event_text: str) -> Optional[str]:
    for kw, emo in EVENT_EMOTION_MAP.items():
        if kw in event_text:
            return emo
    return None
 

def extract_emotion_gemini(event_text: str) -> Optional[str]:
    system_msg = (
        "너는 이벤트/기념일 설명을 읽고 대표 감정을 한국어 단어 1개로만 답한다. "
        "가능한 답: 기쁨, 사랑, 감사, 슬픔, 설렘. 다른 말/문장/기호/마침표 금지."
    )
    prompt = f"기념일: {event_text}\n출력:"
    try:
        model = genai.GenerativeModel("gemini-1.5-pro")
        resp = model.generate_content(
            [system_msg, prompt],
            generation_config=genai.GenerationConfig(temperature=0.2, max_output_tokens=4)
        )
        if not resp or not resp.text:
            return None
        ans = resp.text.strip()
        ans = re.sub(r"[^\w가-힣]", "", ans)  # 불필요 문자 제거
        return ans if ans in {"기쁨","사랑","감사","슬픔","설렘"} else None
    except Exception:
        return None
        return None

def extract_emotion(event_text: str) -> str:
    # 1순위 규칙, 실패 시 2순위 Gemini, 최종 기본값
    return extract_emotion_rule(event_text) or extract_emotion_gemini(event_text) or "기쁨"

# --- 3) 감정 → 색상 추천 (퍼센트 내림차순) ---
def recommend_colors_by_emotion(emotion: str, top_k: int = 2) -> list[str]:
    matched = []
    for color, meta in COLOR_DB.items():
        if emotion in meta.get("emotion", []):
            matched.append((color, meta.get("percentage", 0)))
    matched.sort(key=lambda x: x[1], reverse=True)
    return [c for c, _ in matched[:top_k]] or ["white"]

# --- 4) 계절 보정 ---
SEASON_TONE_MAP = {
    "봄": {"yellow": "pastel yellow", "orange": "pastel orange", "red": "pastel red", "pink": "pastel pink"},
    "여름": {"blue": "sky blue", "white": "pure white", "green": "mint green"},
    "가을": {"orange": "burnt orange", "brown": "warm brown", "yellow": "mustard yellow"},
    "겨울": {"red": "deep red", "green": "pine green", "black": "charcoal black", "white": "snow white"},
}

def get_season(dt: datetime) -> str:
    m = dt.month
    if 3 <= m <= 5:  return "봄"
    if 6 <= m <= 8:  return "여름"
    if 9 <= m <= 11: return "가을"
    return "겨울"

def adjust_colors_for_season(colors: list[str], event_date: datetime) -> list[str]:
    season = get_season(event_date)
    tone_map = SEASON_TONE_MAP.get(season, {})
    return [tone_map.get(c, c) for c in colors]

# --- 5) (옵션) Gemini로 보색/유사색 3개 더 제안 ---
def expand_palette_with_gemini(base_colors: list[str]) -> list[str]:
    """
    입력 색과 잘 어울리는 색 3가지를 한국어 색 이름 또는 HEX 없이 이름만 반환.
    (colorhunt/mycolor.space 연동 전 임시 보완)
    """
    system_msg = (
        "너는 색채 조합 전문가다. 입력 색상과 조화를 이루는 보색/유사색/무채색 포인트 중 "
        "3가지를 한국어 색 이름만 쉼표로 제시해라. HEX/설명 문장/기호 금지."
    )
    prompt = f"입력 색상: {', '.join(base_colors)}\n출력(3개, 쉼표로만):"
    try:
        model = genai.GenerativeModel("gemini-1.5-pro")
        resp = model.generate_content(
            [system_msg, prompt],
            generation_config=genai.GenerationConfig(temperature=0.5, max_output_tokens=16)
        )
        if not resp or not resp.text:
            return []
        # "네이비, 아이보리, 올리브" → 리스트
        items = [x.strip() for x in resp.text.split(",")]
        items = [x for x in items if x]  # 빈 값 제거
        return items[:3]
    except Exception:
        return []
from expand_palette import expand_colors_from_external, preview_links_for_user
# --- 6) 전체 파이프라인 ---
def recommend_colors_for_event(event_text: str, event_date: datetime, top_k_base: int = 2, expand: bool = True):
    emotion = extract_emotion(event_text)
    base_colors = recommend_colors_by_emotion(emotion, top_k=top_k_base)
    seasonal = adjust_colors_for_season(base_colors, event_date)
    seed_bundle = expand_colors_from_external(seasonal, use_colormind=True)
    extra = expand_palette_with_gemini(seasonal) if expand else []
    print("기본(계절 보정) HEX:", seed_bundle["base_hex"])
    print("보색 3:", seed_bundle["complement"])
    print("유사색 3:", seed_bundle["analogous"])
    print("Colormind 팔레트(옵션):", seed_bundle["colormind"])
    return {
        "emotion": emotion,
        "base_colors": base_colors,
        "seasonal_colors": seasonal,
        "extra_colors": extra
    }

# --- 7) 사용 예시 ---
if __name__ == "__main__":
    tests = [
        ("민지 졸업식", datetime(2025, 4, 10)),      # 봄, 기쁨
        ("결혼 10주년 파티", datetime(2025, 12, 5)),  # 겨울, 사랑
        ("아빠 제사", datetime(2025, 9, 20)),        # 가을, 슬픔
        ("동료 승진 축하 겸 회식", datetime(2025, 7, 2)), # 여름, (규칙X→Gemini)
    ]
    for text, dt in tests:
        rec = recommend_colors_for_event(text, dt, top_k_base=2, expand=True)
        print(f"\n[{text} / {dt.date()}]")
        print("감정:", rec["emotion"])
        print("기본 색:", ", ".join(rec["base_colors"]))
        print("계절 보정:", ", ".join(rec["seasonal_colors"]))
        if rec["extra_colors"]:
            print("보완 색(3):", ", ".join(rec["extra_colors"]))



[민지 졸업식 / 2025-04-10]
감정: 기쁨
기본 색: yellow, orange
계절 보정: pastel yellow, pastel orange
보완 색(3): 연보라, 살구색, 아이보리

[결혼 10주년 파티 / 2025-12-05]
감정: 사랑
기본 색: red, pink
계절 보정: deep red, pink
보완 색(3): 진분홍, 연분홍, 회색

[아빠 제사 / 2025-09-20]
감정: 슬픔
기본 색: black
계절 보정: black
보완 색(3): 연백색, 회색, 진회색

[동료 승진 축하 겸 회식 / 2025-07-02]
감정: 기쁨
기본 색: yellow, orange
계절 보정: yellow, orange
보완 색(3): 진노랑, 황토색, 회색


In [ ]:
import os
import json
import re
import chromadb
from sentence_transformers import SentenceTransformer
import google.generativeai as genai
from dotenv import load_dotenv
load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))  # 여기에 실제 키 입력

with open("coloremotion.json", encoding="utf-8") as f:
    db = json.load(f)

emb_model = SentenceTransformer("BAAI/bge-small-en-v1.5")
chroma = chromadb.Client()
try:
    col = chroma.get_collection("color_psych")
except:
    col = chroma.create_collection("color_psych")

try:
    col.delete()
    col = chroma.create_collection("color_psych")
except:
    pass

docs = []
for color, meta in db.items():
    text = f"{color} 색은 {', '.join(meta['emotion'])} 감정을 주며 " \
           f"{', '.join(meta['associations'])} 느낌이 있다."
    docs.append({"id": color, "text": text})

for d in docs:
    emb = emb_model.encode(d["text"]).tolist()
    col.add(ids=[d["id"]], embeddings=[emb], documents=[d["text"]])

def extract_explicit_colors(text: str):
    lower = text.lower()
    return [c for c in db.keys() if c in lower]

def recommend_colors_gemini(user_text: str, top_k: int = 3) -> str:
    explicit = extract_explicit_colors(user_text)
    if explicit:
        return ", ".join(explicit[:2])
    q_emb = emb_model.encode(user_text).tolist()
    hits = col.query(query_embeddings=[q_emb], n_results=top_k)
    context = "\n".join(hits["documents"][0])

    system_msg = (
        "너는 색상 심리학 기반 꽃다발 색상 추천 전문가야. "
        "아래 색상 심리 정보를 참고하고, 사용자 요청에 어울리는 색상 두 가지만 "
        "한국어 단답형으로 추천해줘. 쉼표로 구분하고 이유는 쓰지 마."
    )
    prompt = f"색상 심리 정보:\n{context}\n\n사용자 요청: \"{user_text}\"\n출력:"

    model = genai.GenerativeModel("gemini-1.5-pro") 
    response = model.generate_content(
        [system_msg, prompt],
        generation_config=genai.types.GenerationConfig(
            temperature=0.4,
            max_output_tokens=32
        )
    )

    return re.sub(r"[.\n]+", "", response.text).strip()

if __name__ == "__main__":
    samples = [
        "나 지금 멘탈 나가서 꽃이 받고 싶어",
        "친구 곧 졸업이라 축하해주고 싶어"
    ]
    for text in samples:
        print(f"입력: {text}")
        print(f"추천: {recommend_colors_gemini(text)}\n")


KeyboardInterrupt: 